<a href="https://colab.research.google.com/github/mtomcanyi/Hackhealth2021-Challenge7/blob/main/ZCase_DataLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data Loading

The original data was not a valid JSON dataset, as it was not an array.
We turned it into array and loaded the data into dataframe


In [207]:
import pandas as pd

jsonDF = pd.read_json('/content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/export-commas.json')
jsonDF

,caseId,version,lastStateEventSequence,lastTimelineEventSequence,dispatchingAccountId,dispatchingId,dispatchingTs,createdTs,dispatchingName,destination__geoLocation,closedTs,resolution,attachments__count,media__count,ambulanceLocation__count,roomTeam__count,ambulanceLocation__geoPath,caseAddress__street,caseAddress__number,caseAddress__city,caseAddress__country,caseTeam__name,caseTeam__type,caseTeam__ambulancePhoneNumber,dispatchingTeam__name,dispatchingTeam__dispatcherPhoneNumber,destination__hospitalId,destination__hospitalColor,destination__wardId,destination__wardColor,destination__hospitalName,destination__wardName,ambulanceLocation__first__geoLocation,ambulanceLocation__first__dispatchingEtaTs,ambulanceLocation__first__locationTs,ambulanceLocation__last__geoLocation,ambulanceLocation__last__dispatchingEtaTs,ambulanceLocation__last__locationTs,patient__urgency,patient__classification,...,patient__birthNumber,destinationApproval__status,destinationApproval__requestTs,destinationApproval__responseTs,destinationApproval__refusalReason,dispatchingTs__minutes,createdTs__minutes,closedTs__minutes,ambulanceLocation__first__dispatchingEtaTs__minutes,ambulanceLocation__first__locationTs__minutes,ambulanceLocation__last__dispatchingEtaTs__minutes,ambulanceLocation__last__locationTs__minutes,destinationApproval__requestTs__minutes,destinationApproval__responseTs__minutes,ambulanceLocation__last__dispatchingArrivalTs,ambulanceLocation__last__dispatchingArrivalTs__minutes,patient__birthDate,patient__birthYear,ambulanceLocation__first__dispatchingArrivalTs,ambulanceLocation__first__dispatchingArrivalTs__minutes,destination__dispatchingWardId,patient__age__value,patient__age,destination__dispatchingName,caseAddress__freeText,caseTeam__caseCode,caseTeam__ambulanceCode,destination__destinationApprovalSupported,destination__overriddenWardId,destination__dispatchingSpecialization,patient__naca__value,patient__naca__description,patient__diagnosis__value,patient__diagnosis__description,patient__bloodPressure__sys,patient__bloodPressure__dia,patient__spO2,patient__covid__value,patient__covid__description,patient__gcs__sum
0,dbe02241-e6f0-464e-a98c-46fbe5ed5a35,157,71,8,dispatching:USZSSK,66A9A33F-A8E4-41E1-83A9-778D5423899D,2020-08-31T15:33:45.000Z,2020-08-31T16:12:01.591Z,"ZZS Středočeského kraje, p.o.",POINT (14.3396786 50.0731031),2020-08-31T16:55:52.014Z,ADMISSION,0,0,49,11,"LINESTRING (14.0630896164407 49.9476117924476,...",Ke Kosovu,33,Beroun,CZ,Beroun RZP,RZP,4.207311e+11,"Marušák Martin, Bc.",420155,hospital:MOTOL,1.0,ward:MOTOL:OTHER,1.0,Nemocnice Motol,Motol - Ostatní,POINT (14.0630896164407 49.9476117924476),2020-08-31T16:35:22.839Z,2020-08-31T16:12:11.000Z,POINT (14.3399621152274 50.073666219246),2020-08-31T16:44:05.969Z,2020-08-31T16:37:41.000Z,N3,VERTIGO,...,9.988771e+09,GRANTED,2020-08-31T16:12:01.690Z,2020-08-31T16:12:08.975Z,None,-38,38,44.0,23.0,0.0,6.0,26.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,425901a7-9d32-4daf-ad5b-e3990bc87d78,180,97,7,dispatching:USZSSK,55B444F8-36F3-42CD-B5E5-086CA8ADFE82,2020-08-31T15:51:47.000Z,2020-08-31T16:12:28.315Z,"ZZS Středočeského kraje, p.o.",POINT (14.4575 50.0275),2020-08-31T17:47:04.386Z,ADMISSION,0,0,84,10,"LINESTRING (15.1388673476618 49.6534466972465,...",Zahradní,203,Dolní Kralovice,CZ,Vlašim RZP 1,RZP,4.206052e+11,"Marušák Martin, Bc.",420155,hospital:IKEM,1.0,ward:IKEM:KONTAKT,1.0,IKEM,IKEM Kontaktní místo,POINT (15.1388673476618 49.6534466972465),2020-08-31T17:08:25.045Z,2020-08-31T16:12:43.000Z,POINT (14.6792367197954 49.7870240030342),NaN,2020-08-31T17:05:55.000Z,N3,ÚRAZ,...,9.988771e+09,GRANTED,2020-08-31T16:12:28.399Z,2020-08-31T16:12:37.591Z,None,-21,21,95.0,56.0,0.0,NaN,53.0,0.0,0.0,2020-08-31T17:05:51.000Z,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7a1bd672-a496-4831-ad1c-2137940a331a,115,44,5,dispatching:USZSSK,0A190D51-6EED-4C82-AB12-CC79E1

## Narrowing down the data

We can drop unnecessesary columns

In [208]:
# Drop useless columns

narrowDF = jsonDF.drop(columns=[
  "caseId",
  "caseTeam__ambulancePhoneNumber",
  "patient__birthNumber",
  "ambulanceLocation__first__dispatchingArrivalTs",
  "ambulanceLocation__first__dispatchingArrivalTs__minutes",
  "destination__destinationApprovalSupported",
  "destination__hospitalColor",
  "destination__wardColor",
  "dispatchingTeam__dispatcherPhoneNumber",
  "dispatchingTs__minutes", # negative value of createdTs__minutes
  ])


## Narrowing down data

Original dataset had 84 columns

In [209]:
narrowDF.shape

(48975, 74)

## Test data present
When looking at Dispatch centers we found what seems test entries.
Notice the value  `Test dispatching`

In [210]:
dispatchNamesUniqueDF = pd.DataFrame(narrowDF["dispatchingName"].unique(), columns=["Unique entries"])
dispatchNamesUniqueDF

,Unique entries
0,"ZZS Středočeského kraje, p.o."
1,Test dispatching
2,ZZS hl.m.Prahy
3,ZZS Moravskoslezského kraje
4,ZZS Olomouckého kraje


## Removing entries
Let's remove the 2 problematic records

In [212]:
# Delete test data entries
#narrowDF.loc[narrowDF["dispatchingName"] == "Test dispatching"]
#noTestDataDF = narrowDF.drop([53,5063])

noTestDataDF = narrowDF.loc[narrowDF["dispatchingName"] != "Test dispatching"]
noTestDataDF.shape

# Explore the two drop records
#noTestDataPrint = pd.concat([noTestDataDF[50:55],noTestDataDF[5060:5065]])
#noTestDataPrint

(48973, 74)

## More test data
There are more test data records found in `patient__classification`
which needs to be removed

In [200]:
patientClassificationTestDataIdx = noTestDataDF["patient__classification"].str.contains("test", case=False, na=False)
noTestDataDF[patientClassificationTestDataIdx][["patient__classification","dispatchingId"]]

,patient__classification,dispatchingId
3010,Testflight testování,mv200
3329,Testflight testování,mv300
4563,Testovací infarkt,mv001
4566,Test case,mv002
4603,Test case 2,mv003
6950,Testovací případ,mv001
7011,Testovací výjezd,mv111
7013,2 - Testovací výjezd,mv112
7014,2 - Testovací výjezd,mv113
7015,4 - Testovací výjezd,mv114


In [213]:
# Invert the bool vector so we keep only valid data
noTestDataDF = noTestDataDF.loc[~patientClassificationTestDataIdx ]
noTestDataDF.shape

(48927, 74)

## Dubious record
This record seems valid, but destination approval was rejected with reason `pokus`. We are not deleting it for now, because it seems valid.

In [191]:
# not deleting for now
noTestDataDF.loc[noTestDataDF["destinationApproval__refusalReason"].str.contains("pokus", case=False, na=False)]

,version,lastStateEventSequence,lastTimelineEventSequence,dispatchingAccountId,dispatchingId,dispatchingTs,createdTs,dispatchingName,destination__geoLocation,closedTs,resolution,attachments__count,media__count,ambulanceLocation__count,roomTeam__count,ambulanceLocation__geoPath,caseAddress__street,caseAddress__number,caseAddress__city,caseAddress__country,caseTeam__name,caseTeam__type,dispatchingTeam__name,destination__hospitalId,destination__wardId,destination__hospitalName,destination__wardName,ambulanceLocation__first__geoLocation,ambulanceLocation__first__dispatchingEtaTs,ambulanceLocation__first__locationTs,ambulanceLocation__last__geoLocation,ambulanceLocation__last__dispatchingEtaTs,ambulanceLocation__last__locationTs,patient__urgency,patient__classification,patient__comment,patient__firstName,patient__lastName,patient__sex,destinationApproval__status,destinationApproval__requestTs,destinationApproval__responseTs,destinationApproval__refusalReason,createdTs__minutes,closedTs__minutes,ambulanceLocation__first__dispatchingEtaTs__minutes,ambulanceLocation__first__locationTs__minutes,ambulanceLocation__last__dispatchingEtaTs__minutes,ambulanceLocation__last__locationTs__minutes,destinationApproval__requestTs__minutes,destinationApproval__responseTs__minutes,ambulanceLocation__last__dispatchingArrivalTs,ambulanceLocation__last__dispatchingArrivalTs__minutes,patient__birthDate,patient__birthYear,destination__dispatchingWardId,patient__age__value,patient__age,destination__dispatchingName,caseAddress__freeText,caseTeam__caseCode,caseTeam__ambulanceCode,destination__overriddenWardId,destination__dispatchingSpecialization,patient__naca__value,patient__naca__description,patient__diagnosis__value,patient__diagnosis__description,patient__bloodPressure__sys,patient__bloodPressure__dia,patient__spO2,patient__covid__value,patient__covid__description,patient__gcs__sum
2097,152,144,4,dispatching:USZSSK,F464CD5D-AA4E-4E2B-BF1A-47BFD2571DD4,2020-10-01T08:46:23.000Z,2020-10-01T08:51:12.312Z,"ZZS Středočeského kraje, p.o.",POINT (14.354306 50.074998),2020-10-01T10:29:52.571Z,ADMISSION,0,0,119,15,"LINESTRING (13.9026696979022 49.8415499086013,...",K Nemocnici,1106,Hořovice,CZ,Hořovice RLP,RLP,"Vajcová Jana, Bc.",hospital:HOMOLKA,ward:HOMOLKA:KARDIO,Nemocnice Na Homolce,Homolka - Koronární jednotka,POINT (13.9026696979022 49.8415499086013),2020-10-01T09:23:13.234Z,2020-10-01T08:51:14.000Z,POINT (14.3540925755365 50.0748542253845),NaN,2020-10-01T09:54:13.000Z,N2,SEKUNDÁRNÍ TRANSPORT AKUTNÍ,Anonymized. Case bc2ceff4-14a9-4350-8aef-42228...,Rainold,Morávek,M,REFUSED,2020-10-01T08:51:12.464Z,2020-10-01T10:01:32.282Z,Pokus,5,99.0,32.0,0.0,NaN,63.0,0.0,70.0,2020-10-01T09:33:15.000Z,42.0,NaN,NaN,1053.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Test data in hospitalId field
Remove the entries from data set


In [202]:
hospTestIdx = noTestDataDF["destination__hospitalId"].str.contains("hospital:TEST", case=False, na=False)
noTestDataDF = noTestDataDF.loc[~hospTestIdx]
noTestDataDF.shape

(48904, 74)

## Dubious years

Found values `>2021` and `<1910`


In [216]:
# Exploration of those records
#
# suspiciousYear = pd.concat([
#                             noTestDataDF[noTestDataDF["patient__birthYear"]>2021],
#                             noTestDataDF[noTestDataDF["patient__birthYear"]<1910]
# ])


after2021Idx = noTestDataDF["patient__birthYear"]>2021
before1900Idx = noTestDataDF["patient__birthYear"]<1910

# Merge the above boolean indexes
invalidDataIdx = after2021Idx | before1900Idx

noTestDataDF = noTestDataDF.loc[~invalidDataIdx]
noTestDataDF.shape

(48883, 74)

In [217]:
# Joke sent to #random channel
# noTestDataDF[noTestDataDF["patient__birthYear"] < 1921][["patient__firstName","patient__lastName","patient__birthYear","patient__age","patient__age__value","caseAddress__city"]]

noTestDataDF["patient__birthYear"].describe()

count    40338.000000
mean      1961.643636
std         23.210964
min       1913.000000
25%       1943.000000
50%       1956.000000
75%       1978.000000
max       2021.000000
Name: patient__birthYear, dtype: float64

In [229]:
for column in ["dispatchingAccountId",
               "dispatchingName",
               "resolution", 
               "caseTeam__name",
               "caseTeam__type", 
               "destination__hospitalId",
               "patient__urgency",
               "patient__classification",
               "patient__sex",
               "destinationApproval__status",
               "destinationApproval__refusalReason",
               "patient__birthYear"]:
  print(f"---------[{column}]--------------")
  #print(noTestDataDF[column].unique())
  print(jsonDF[column].unique())

# caseTeam__name contains <img src=q
# caseAddress__city ... many entries

---------[dispatchingAccountId]--------------
['dispatching:USZSSK' 'dispatching:TEST' 'dispatching:ZZSHMP'
 'dispatching:ZZSMSK' 'dispatching:ZZSOK']
---------[dispatchingName]--------------
['ZZS Středočeského kraje, p.o.' 'Test dispatching' 'ZZS hl.m.Prahy'
 'ZZS Moravskoslezského kraje' 'ZZS Olomouckého kraje']
---------[resolution]--------------
['ADMISSION' nan 'UNSPECIFIED']
---------[caseTeam__name]--------------
['Beroun RZP' 'Vlašim RZP 1' 'Nové Strašecí RZP' 'Zdice RZP'
 'Brandýs RZP 2' 'Zdiby RZP' 'Kladno RZP 2' 'Řevnice RLP' 'Český Brod RLP'
 'Příbram RZP 1' 'Lysá n/L  RZP' 'Brandýs RZP 1' 'Kralupy RZP 1'
 'Neratovice RLP' 'Kladno RZP 1' 'Ml. Boleslav RLP' 'Zbraslav RZP'
 'Dobříš RZP' 'Mělník RZP 2' 'Jesenice RLP' 'Vlašim RZP 2' 'Kladno RZP 3'
 'Hostivice RZP' 'Řevnice záložní' 'Davle RZP' 'Rakovník RZP2'
 'Říčany RZP 1' 'Rakovník RZP1' 'Příbram RLP' 'Rychla VIII' 'Říčany RZP 2'
 'Slaný RZP 2' 'Mělník RZP 1' 'Roztoky Pr. RZP' 'Kolín RZP 2' 'Benešov 2'
 'Hořovice RLP' 'Ml. 

In [222]:
noTestDataDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48883 entries, 0 to 48974
Data columns (total 74 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   version                                                 48883 non-null  int64  
 1   lastStateEventSequence                                  48883 non-null  int64  
 2   lastTimelineEventSequence                               48883 non-null  int64  
 3   dispatchingAccountId                                    48883 non-null  object 
 4   dispatchingId                                           48883 non-null  object 
 5   dispatchingTs                                           48883 non-null  object 
 6   createdTs                                               48883 non-null  object 
 7   dispatchingName                                         48883 non-null  object 
 8   destination__geoLocation            

## Export data to `arrivals-clean.csv`


In [225]:
noTestDataDF.to_csv('/content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/arrivals-clean.csv')

In [227]:
! wc -l /content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/arrivals-clean.csv

48884 /content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/arrivals-clean.csv


## Can we drop dispatchingTs__minutes?
The column `dispatchingTs__minutes` seems to be just negative value of `createdTs__minutes` apart of 4 records (... where there is 1 minute difference).

So YES, the column can be dropped

## SQL Injection?
One test record contains values indicating possible SQL injection attack

In [232]:
jsonDF.loc[jsonDF["caseTeam__name"].str.contains("onerror=alert", case=False, na=False)]

,caseId,version,lastStateEventSequence,lastTimelineEventSequence,dispatchingAccountId,dispatchingId,dispatchingTs,createdTs,dispatchingName,destination__geoLocation,closedTs,resolution,attachments__count,media__count,ambulanceLocation__count,roomTeam__count,ambulanceLocation__geoPath,caseAddress__street,caseAddress__number,caseAddress__city,caseAddress__country,caseTeam__name,caseTeam__type,caseTeam__ambulancePhoneNumber,dispatchingTeam__name,dispatchingTeam__dispatcherPhoneNumber,destination__hospitalId,destination__hospitalColor,destination__wardId,destination__wardColor,destination__hospitalName,destination__wardName,ambulanceLocation__first__geoLocation,ambulanceLocation__first__dispatchingEtaTs,ambulanceLocation__first__locationTs,ambulanceLocation__last__geoLocation,ambulanceLocation__last__dispatchingEtaTs,ambulanceLocation__last__locationTs,patient__urgency,patient__classification,...,patient__birthNumber,destinationApproval__status,destinationApproval__requestTs,destinationApproval__responseTs,destinationApproval__refusalReason,dispatchingTs__minutes,createdTs__minutes,closedTs__minutes,ambulanceLocation__first__dispatchingEtaTs__minutes,ambulanceLocation__first__locationTs__minutes,ambulanceLocation__last__dispatchingEtaTs__minutes,ambulanceLocation__last__locationTs__minutes,destinationApproval__requestTs__minutes,destinationApproval__responseTs__minutes,ambulanceLocation__last__dispatchingArrivalTs,ambulanceLocation__last__dispatchingArrivalTs__minutes,patient__birthDate,patient__birthYear,ambulanceLocation__first__dispatchingArrivalTs,ambulanceLocation__first__dispatchingArrivalTs__minutes,destination__dispatchingWardId,patient__age__value,patient__age,destination__dispatchingName,caseAddress__freeText,caseTeam__caseCode,caseTeam__ambulanceCode,destination__destinationApprovalSupported,destination__overriddenWardId,destination__dispatchingSpecialization,patient__naca__value,patient__naca__description,patient__diagnosis__value,patient__diagnosis__description,patient__bloodPressure__sys,patient__bloodPressure__dia,patient__spO2,patient__covid__value,patient__covid__description,patient__gcs__sum
5063,25c83083-b447-482d-9539-99964434b95c,13,18,5,dispatching:TEST,NEW_CASE_01,2020-02-02T08:24:18.557Z,2020-11-19T07:24:37.671Z,Test dispatching,POINT (14.4575 50.0275),NaN,NaN,0,5,0,1,NaN,Street﹤img src=q onerror=alert(1)﹥'{,14﹤img src=q onerror=alert(1)﹥'{,Praha﹤img src=q onerror=alert(1)﹥'{,QQ,﹤img src=q onerror=alert(1)﹥'{,RZP,4.201235e+11,﹤img src=q onerror=alert(1)﹥'{,420123456789,hospital:IKEM,1.0,ward:IKEM:TEST,1.0,IKEM,IKEM IT oddělení,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-418980,418980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#jsonDF[["dispatchingTs","createdTs","dispatchingTs__minutes","createdTs__minutes"]][:10]

jsonDF[jsonDF["dispatchingTs__minutes"] != jsonDF["createdTs__minutes"]*-1][["dispatchingTs__minutes","createdTs__minutes"]]


In [ ]:
#headDF = jsonDF[:50]
#headDF.to_csv('/content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/head-50.csv')

#jsonDF.to_csv('/content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/export-commas.csv')

#jsonDF["caseTeam__type"].unique()
#jsonDF[[jsonDF["caseID"] =="dbe02241-e6f0-464e-a98c-46fbe5ed5a35" ]]


In [ ]:
#csvDF = pd.read_csv('/content/drive/MyDrive/Hackhealth2021/Challenge7-zCase/ArrivalData/export-commas.csv')